***GENERATED CODE FOR telcochurndatapipeline PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import pandas as pd
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        hdfs = HDFileSystem(host=hdfs_server, port=hdfs_port)
        #hdfs = HDFileSystem(host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            dfPd = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def calculateFormula(df, functionsData, applyOn):
    dfcp = df
    for functionData in functionsData:
        if functionData['useNext']:
            if 'dataPrepColumn_Inprocess' in list(dfcp.columns):
                dfcp['dataPrepColumn_Inprocess'] = calculateEquation(
                    dfcp, functionData['operator'], functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
            else:
                dfcp['dataPrepColumn_Inprocess'] = calculateEquation(
                    dfcp, functionData['operator'], functionData['operand1'], functionData['operand2'])
        else:
            if 'dataPrepColumn_Inprocess' in list(dfcp.columns):
                if functionData['outputColumn'] == "newColumn":
                    newColumnName = "New_" + \
                        functionData['operand1']+"_" + \
                        functionData['operator']+"_"+functionData['operand2']
                    dfcp[newColumnName] = calculateEquation(dfcp, functionData['operator'],
                                                            functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
                else:
                    dfcp[functionData['outputColumn']] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
                dfcp.drop('dataPrepColumn_Inprocess', axis=1)
            else:
                if functionData['outputColumn'] == "newColumn":
                    newColumnName = "New_" + \
                        functionData['operand1']+"_" + \
                        functionData['operator']+"_"+functionData['operand2']
                    dfcp[newColumnName] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['operand1'], functionData['operand2'])
                else:
                    dfcp[functionData['outputColumn']] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['operand1'], functionData['operand2'])
    return dfcp


def calculateEquation(df, operator, columnA, columnB):
    if (operator == "add"):
        return df[columnA].add(df[columnB])
    elif (operator == "sub"):
        return df[columnA].sub(df[columnB])
    elif (operator == "mul"):
        return df[columnA].mul(df[columnB])
    elif (operator == "div"):
        return df[columnA].div(df[columnB])
    elif (operator == "mod"):
        return df[columnA].mod(df[columnB])


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run telcochurndatapipelineHooks.ipynb
try:
	#sourcePreExecutionHook()

	telcocutomerchurntrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1695815396284_Telco_Customer_Churn_Train.csv', 'filename': 'Telco_Customer_Churn_Train.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/Retail And Marketing/Telco Churn/Telco_Customer_Churn_Train.csv', 'viewFileName': 'Telco_Customer_Churn_Train.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PERFORMING OPERATIONS***

In [ ]:
#%run telcochurndatapipelineHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(telcocutomerchurntrain,spark,json.dumps( {"url": "/FileStore/platform/testdata/1695815396284_Telco_Customer_Churn_Train.csv", "source_attributes": {}, "DataPrepFile": "/FileStore/platform/testdata/1695815396284_Telco_Customer_Churn_Train.csv", "data_source": "platfiles", "startListenerOnly": 1, "dateColumnNames": [], "FilePath": "/FileStore/platform/extra/654d2a3bbb5c80aef638770e1699556223/0part.csv", "requestRatio": 14.0, "totalRows": 7043, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 22, "numberOfRows": 7043, "duplicateRowCount": 0, "stats": [{"column": "customerID", "alias": "customerID", "generated": 0, "type": "String", "max": "9995-HOTOH", "min": "0002-ORFBO", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "gender", "alias": "gender", "generated": 0, "type": "String", "max": "Male", "min": "Female", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "SeniorCitizen", "alias": "SeniorCitizen", "generated": 0, "type": "numeric", "max": 1, "min": 0, "mean": 0.1621468124378816, "missing": 0.0, "stddev": 0.37, "outliers": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], "validation": []}, {"column": "Partner", "alias": "Partner", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "Dependents", "alias": "Dependents", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "tenure", "alias": "tenure", "generated": 0, "type": "numeric", "max": 72, "min": 0, "mean": 32.37114865824223, "missing": 0.0, "stddev": 24.56, "outliers": [], "validation": []}, {"column": "PhoneService", "alias": "PhoneService", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "MultipleLines", "alias": "MultipleLines", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "InternetService", "alias": "InternetService", "generated": 0, "type": "String", "max": "No", "min": "DSL", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "OnlineSecurity", "alias": "OnlineSecurity", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "OnlineBackup", "alias": "OnlineBackup", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "DeviceProtection", "alias": "DeviceProtection", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "TechSupport", "alias": "TechSupport", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "StreamingTV", "alias": "StreamingTV", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "StreamingMovies", "alias": "StreamingMovies", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "Contract", "alias": "Contract", "generated": 0, "type": "String", "max": "Two year", "min": "Month-to-month", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "PaperlessBilling", "alias": "PaperlessBilling", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "PaymentMethod", "alias": "PaymentMethod", "generated": 0, "type": "String", "max": "Mailed check", "min": "Bank transfer (automatic)", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "MonthlyCharges", "alias": "MonthlyCharges", "generated": 0, "type": "real", "max": 118.75, "min": 18.25, "mean": 64.76169246059918, "missing": 0.0, "stddev": 30.09, "outliers": [], "validation": []}, {"column": "TotalCharges", "alias": "TotalCharges", "generated": 0, "type": "real", "max": 8684.8, "min": 18.8, "mean": 2283.3004408418656, "missing": 0.2, "stddev": 2266.77, "outliers": [8129.3, 8041.65, 8126.65, 8468.2, 8404.9, 8399.15, 8248.5, 8220.4, 8071.05, 8306.05, 8196.4, 8065.65, 8476.5, 8310.55, 8046.85, 8333.95, 8093.15, 8317.95, 8127.6, 8375.05, 8100.55, 8078.1, 8086.4, 8477.7, 8477.6, 8564.75, 8124.2, 8529.5, 8058.85, 8164.1, 8425.3, 8496.7, 8405.0, 8277.05, 8165.1, 8182.85, 8182.75, 8308.9, 8240.85, 8061.5, 8297.5, 8109.8, 8425.15, 8058.55, 8436.25, 8349.45, 8456.75, 8443.7, 8312.4, 8424.9, 8100.25, 8672.45, 8684.8, 8332.15, 8312.75, 8289.2, 8166.8, 8075.35, 8594.4, 8244.3, 8543.25, 8331.95, 8250.0, 8192.6, 8547.15, 8175.9, 8309.55, 8337.45, 8152.3, 8349.7, 8670.1], "validation": []}, {"column": "Churn", "alias": "Churn", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "TotalCharges1", "alias": "TotalCharges1", "generated": 1, "type": "real", "max": 8802.599999999999, "min": 37.6, "mean": 2348.0986490329924, "missing": 0.2, "stddev": 2286.47, "outliers": [8239.0, 8153.9, 8239.25, 8582.25, 8520.949999999999, 8514.9, 8361.6, 8336.0, 8182.95, 8420.599999999999, 8311.35, 8176.299999999999, 8592.3, 8426.65, 8159.25, 8448.6, 8207.15, 8432.800000000001, 8243.15, 8489.5, 8215.0, 8193.25, 8200.5, 8594.550000000001, 8596.25, 8681.0, 8237.85, 8646.65, 8169.3, 8277.5, 8540.849999999999, 8611.6, 8520.05, 8393.8, 8274.65, 8298.85, 8296.4, 8426.5, 8153.15, 8351.75, 8174.4, 8413.55, 8219.05, 8540.65, 8172.85, 8553.6, 8464.6, 8573.55, 8559.45, 8427.949999999999, 8540.699999999999, 8214.85, 8791.2, 8802.599999999999, 8447.949999999999, 8428.25, 8403.25, 8280.45, 8185.1, 8711.35, 8358.599999999999, 8659.65, 8446.45, 8365.15, 8305.35, 8145.05, 8665.35, 8289.9, 8425.849999999999, 8454.050000000001, 8268.85, 8464.85, 8787.6], "validation": []}]}, "predictionPowerScore": [{"Churn": 1.0, "Contract": 0.0, "Dependents": 0.0, "DeviceProtection": 0.0, "InternetService": 0.0, "MonthlyCharges": 0.1045036141, "MultipleLines": 0.0, "OnlineBackup": 0.0, "OnlineSecurity": 0.0, "PaperlessBilling": 0.0, "Partner": 0.0, "PaymentMethod": 0.0, "PhoneService": 0.0, "SeniorCitizen": 0.0, "StreamingMovies": 0.0, "StreamingTV": 0.0, "TechSupport": 0.0, "TotalCharges": 0.078636906, "TotalCharges1": 0.0812499187, "customerID": 0.0, "gender": 0.0, "tenure": 0.2316790568}, {"Churn": 0.0, "Contract": 1.0, "Dependents": 0.0, "DeviceProtection": 0.1324787755, "InternetService": 0.1324787755, "MonthlyCharges": 0.1336610312, "MultipleLines": 0.0, "OnlineBackup": 0.1324787755, "OnlineSecurity": 0.1537112968, "PaperlessBilling": 0.0, "Partner": 0.0, "PaymentMethod": 0.1231568454, "PhoneService": 0.0, "SeniorCitizen": 0.0, "StreamingMovies": 0.1324787755, "StreamingTV": 0.1324787755, "TechSupport": 0.2078582575, "TotalCharges": 0.1981644541, "TotalCharges1": 0.1841336905, "customerID": 0.0, "gender": 0.0, "tenure": 0.4088157423}, {"Churn": 0.0, "Contract": 0.0, "Dependents": 1.0, "DeviceProtection": 0.0, "InternetService": 0.0, "MonthlyCharges": 0.0529379812, "MultipleLines": 0.0, "OnlineBackup": 0.0, "OnlineSecurity": 0.0, "PaperlessBilling": 0.0, "Partner": 0.3421008511, "PaymentMethod": 0.0, "PhoneService": 0.0, "SeniorCitizen": 0.0, "StreamingMovies": 0.0, "StreamingTV": 0.0, "TechSupport": 0.0, "TotalCharges": 0.0453188832, "TotalCharges1": 0.0155880179, "customerID": 0.0, "gender": 0.0, "tenure": 0.0}, {"Churn": 0.0508945535, "Contract": 0.2002736777, "Dependents": 0.0, "DeviceProtection": 1.0, "InternetService": 0.2603468184, "MonthlyCharges": 0.4971032173, "MultipleLines": 0.0643494851, "OnlineBackup": 0.5038689797, "OnlineSecurity": 0.5023303075, "PaperlessBilling": 0.0025865337, "Partner": 0.0779447521, "PaymentMethod": 0.2240796305, "PhoneService": 0.0, "SeniorCitizen": 0.0, "StreamingMovies": 0.5735403291, "StreamingTV": 0.5691087246, "TechSupport": 0.5442739101, "TotalCharges": 0.177483329, "TotalCharges1": 0.1897775605, "customerID": 0.0, "gender": 0.0, "tenure": 0.1773312613}, {"Churn": 0.0756726175, "Contract": 0.1128009697, "Dependents": 0.0545757668, "DeviceProtection": 0.2863912193, "InternetService": 1.0, "MonthlyCharges": 0.7769658444, "MultipleLines": 0.1768821428, "OnlineBackup": 0.2863912193, "OnlineSecurity": 0.5273376578, "PaperlessBilling": 0.0823930252, "Partner": 0.0, "PaymentMethod": 0.1015250835, "PhoneService": 0.142028373, "SeniorCitizen": 0.0, "StreamingMovies": 0.4899613408, "StreamingTV": 0.4996716973, "TechSupport": 0.5296214257, "TotalCharges": 0.1950438593, "TotalCharges1": 0.1884838456, "customerID": 0.0, "gender": 0.0, "tenure": 0.0}, {"Churn": 0.0185730084, "Contract": 0.0, "Dependents": 0.0, "DeviceProtection": 0.4640620694, "InternetService": 0.6250831489, "MonthlyCharges": 1.0, "MultipleLines": 0.1718835846, "OnlineBackup": 0.4500768721, "OnlineSecurity": 0.4341602111, "PaperlessBilling": 0.0750736959, "Partner": 0.0, "PaymentMethod": 0.1006047859, "PhoneService": 0.0523489805, "SeniorCitizen": 0.0197274673, "StreamingMovies": 0.5156816942, "StreamingTV": 0.5200358908, "TechSupport": 0.436601225, "TotalCharges": 0.1432123517, "TotalCharges1": 0.1412379151, "customerID": 0.0, "gender": 0.0, "tenure": 0.0168679774}, {"Churn": 0.0, "Contract": 0.0797676586, "Dependents": 0.0, "DeviceProtection": 0.194049849, "InternetService": 0.2501146933, "MonthlyCharges": 0.4240081242, "MultipleLines": 1.0, "OnlineBackup": 0.1929921954, "OnlineSecurity": 0.130171174, "PaperlessBilling": 0.1329666272, "Partner": 0.1363512842, "PaymentMethod": 0.1020275123, "PhoneService": 0.0277673318, "SeniorCitizen": 0.043928772, "StreamingMovies": 0.2347502255, "StreamingTV": 0.2316058164, "TechSupport": 0.130171174, "TotalCharges": 0.2101512738, "TotalCharges1": 0.2203761935, "customerID": 0.0, "gender": 0.0, "tenure": 0.2363117907}, {"Churn": 0.0, "Contract": 0.1500235169, "Dependents": 0.0, "DeviceProtection": 0.5048915826, "InternetService": 0.2727294137, "MonthlyCharges": 0.4747304079, "MultipleLines": 0.0691993944, "OnlineBackup": 1.0, "OnlineSecurity": 0.5053568167, "PaperlessBilling": 0.0052504462, "Partner": 0.0595143067, "PaymentMethod": 0.2209544214, "PhoneService": 0.0, "SeniorCitizen": 0.0, "StreamingMovies": 0.4171948562, "StreamingTV": 0.4754281175, "TechSupport": 0.5083287957, "TotalCharges": 0.2182563092, "TotalCharges1": 0.2140829524, "customerID": 0.0, "gender": 0.0, "tenure": 0.1565974051}, {"Churn": 0.0, "Contract": 0.1592326897, "Dependents": 0.0, "DeviceProtection": 0.3550246723, "InternetService": 0.3550246723, "MonthlyCharges": 0.4839400675, "MultipleLines": 0.0, "OnlineBackup": 0.3550246723, "OnlineSecurity": 1.0, "PaperlessBilling": 0.0877796668, "Partner": 0.0, "PaymentMethod": 0.1514982186, "PhoneService": 0.0, "SeniorCitizen": 0.0, "StreamingMovies": 0.3550246723, "StreamingTV": 0.3550246723, "TechSupport": 0.5664366009, "TotalCharges": 0.1852874633, "TotalCharges1": 0.1645981806, "customerID": 0.0, "gender": 0.0, "tenure": 0.1154553585}, {"Churn": 0.0, "Contract": 0.1338349116, "Dependents": 0.0, "DeviceProtection": 0.2965414953, "InternetService": 0.2965414953, "MonthlyCharges": 0.1720162693, "MultipleLines": 0.0, "OnlineBackup": 0.2965414953, "OnlineSecurity": 0.2965414953, "PaperlessBilling": 1.0, "Partner": 0.0, "PaymentMethod": 0.1831726456, "PhoneService": 0.0, "SeniorCitizen": 0.0, "StreamingMovies": 0.2965414953, "StreamingTV": 0.2965414953, "TechSupport": 0.2965414953, "TotalCharges": 0.0948356859, "TotalCharges1": 0.0875940731, "customerID": 0.0, "gender": 0.0, "tenure": 0.0}, {"Churn": 0.0417174601, "Contract": 0.2603146326, "Dependents": 0.3964116473, "DeviceProtection": 0.1104955386, "InternetService": 0.0, "MonthlyCharges": 0.0307940496, "MultipleLines": 0.1250671197, "OnlineBackup": 0.0792829769, "OnlineSecurity": 0.0737233118, "PaperlessBilling": 0.0, "Partner": 1.0, "PaymentMethod": 0.1514947037, "PhoneService": 0.0, "SeniorCitizen": 0.0, "StreamingMovies": 0.0980271056, "StreamingTV": 0.0965148404, "TechSupport": 0.0618985563, "TotalCharges": 0.126109543, "TotalCharges1": 0.1460848852, "customerID": 0.0, "gender": 0.0, "tenure": 0.2753671602}, {"Churn": 0.0, "Contract": 0.0597291602, "Dependents": 0.0, "DeviceProtection": 0.0526124827, "InternetService": 0.0736883015, "MonthlyCharges": 0.0964886158, "MultipleLines": 0.0, "OnlineBackup": 0.0526124827, "OnlineSecurity": 0.1665444392, "PaperlessBilling": 0.0169743113, "Partner": 0.0, "PaymentMethod": 1.0, "PhoneService": 0.0, "SeniorCitizen": 0.0, "StreamingMovies": 0.0526124827, "StreamingTV": 0.0526124827, "TechSupport": 0.1624651915, "TotalCharges": 0.0384132871, "TotalCharges1": 0.0656984786, "customerID": 0.0, "gender": 0.0, "tenure": 0.081978425}, {"Churn": 0.0, "Contract": 0.0, "Dependents": 0.0, "DeviceProtection": 0.0, "InternetService": 0.0, "MonthlyCharges": 0.3831399318, "MultipleLines": 1.0, "OnlineBackup": 0.0, "OnlineSecurity": 0.0, "PaperlessBilling": 0.0, "Partner": 0.0, "PaymentMethod": 0.0, "PhoneService": 1.0, "SeniorCitizen": 0.0, "StreamingMovies": 0.0, "StreamingTV": 0.0, "TechSupport": 0.0, "TotalCharges": 0.0, "TotalCharges1": 0.0, "customerID": 0.0, "gender": 0.0, "tenure": 0.0}, {"Churn": 0.0, "Contract": 0.0, "Dependents": 0.0, "DeviceProtection": 0.0, "InternetService": 0.0, "MonthlyCharges": 0.0, "MultipleLines": 0.0, "OnlineBackup": 0.0, "OnlineSecurity": 0.0, "PaperlessBilling": 0.0, "Partner": 0.0, "PaymentMethod": 0.0, "PhoneService": 0.0, "SeniorCitizen": 1.0, "StreamingMovies": 0.0, "StreamingTV": 0.0, "TechSupport": 0.0, "TotalCharges": 0.0, "TotalCharges1": 0.0, "customerID": 0.0, "gender": 0.0, "tenure": 0.0}, {"Churn": 0.0, "Contract": 0.1245726967, "Dependents": 0.0, "DeviceProtection": 0.5719356539, "InternetService": 0.4848958001, "MonthlyCharges": 0.6083178079, "MultipleLines": 0.0973177254, "OnlineBackup": 0.4681460108, "OnlineSecurity": 0.4190439604, "PaperlessBilling": 0.0, "Partner": 0.0635067994, "PaymentMethod": 0.0, "PhoneService": 0.0, "SeniorCitizen": 0.0, "StreamingMovies": 1.0, "StreamingTV": 0.6688544494, "TechSupport": 0.4869711533, "TotalCharges": 0.2059264644, "TotalCharges1": 0.233432547, "customerID": 0.0, "gender": 0.0, "tenure": 0.1120807469}, {"Churn": 0.0, "Contract": 0.1241985229, "Dependents": 0.0, "DeviceProtection": 0.5685460301, "InternetService": 0.4959616677, "MonthlyCharges": 0.610206676, "MultipleLines": 0.1005058419, "OnlineBackup": 0.4736687351, "OnlineSecurity": 0.4108718217, "PaperlessBilling": 0.0, "Partner": 0.0644514403, "PaymentMethod": 0.0, "PhoneService": 0.0, "SeniorCitizen": 0.0, "StreamingMovies": 0.6696561612, "StreamingTV": 1.0, "TechSupport": 0.4874527217, "TotalCharges": 0.2296543376, "TotalCharges1": 0.2338597511, "customerID": 0.0, "gender": 0.0, "tenure": 0.1300004092}, {"Churn": 0.0, "Contract": 0.190402535, "Dependents": 0.0, "DeviceProtection": 0.5477869782, "InternetService": 0.3916785201, "MonthlyCharges": 0.479980282, "MultipleLines": 0.0, "OnlineBackup": 0.3540604098, "OnlineSecurity": 0.5723285344, "PaperlessBilling": 0.0843586083, "Partner": 0.0, "PaymentMethod": 0.1549937117, "PhoneService": 0.0, "SeniorCitizen": 0.0, "StreamingMovies": 0.3540604098, "StreamingTV": 0.3540604098, "TechSupport": 1.0, "TotalCharges": 0.1806223076, "TotalCharges1": 0.1736355933, "customerID": 0.0, "gender": 0.0, "tenure": 0.1364004646}, {"Churn": 0.0, "Contract": 0.0742154176, "Dependents": 0.0, "DeviceProtection": 0.1731762429, "InternetService": 0.0651473002, "MonthlyCharges": 0.2266090286, "MultipleLines": 0.0940944064, "OnlineBackup": 0.1602606706, "OnlineSecurity": 0.1129094049, "PaperlessBilling": 0.0, "Partner": 0.0032774794, "PaymentMethod": 0.021745533, "PhoneService": 0.0, "SeniorCitizen": 0.0, "StreamingMovies": 0.1491788656, "StreamingTV": 0.1531873297, "TechSupport": 0.1176828933, "TotalCharges": 1.0, "TotalCharges1": 0.9874370313, "customerID": 0.0, "gender": 0.0, "tenure": 0.5144616861}, {"Churn": 0.0, "Contract": 0.0719746157, "Dependents": 0.0, "DeviceProtection": 0.1766409947, "InternetService": 0.0698779119, "MonthlyCharges": 0.2336419292, "MultipleLines": 0.0953061456, "OnlineBackup": 0.163676234, "OnlineSecurity": 0.1160574705, "PaperlessBilling": 0.0, "Partner": 0.0030675197, "PaymentMethod": 0.0222166901, "PhoneService": 0.0, "SeniorCitizen": 0.0, "StreamingMovies": 0.1534965119, "StreamingTV": 0.1575395793, "TechSupport": 0.1210108821, "TotalCharges": 0.9875916228, "TotalCharges1": 1.0, "customerID": 0.0, "gender": 0.0, "tenure": 0.5047453695}, {"Churn": 0.0, "Contract": 0.0, "Dependents": 0.0, "DeviceProtection": 0.0, "InternetService": 0.0, "MonthlyCharges": 0.0, "MultipleLines": 0.0, "OnlineBackup": 0.0, "OnlineSecurity": 0.0, "PaperlessBilling": 0.0, "Partner": 0.0, "PaymentMethod": 0.0, "PhoneService": 0.0, "SeniorCitizen": 0.0, "StreamingMovies": 0.0, "StreamingTV": 0.0, "TechSupport": 0.0, "TotalCharges": 0.0, "TotalCharges1": 0.0, "customerID": 1.0, "gender": 0.0, "tenure": 0.0}, {"Churn": 0.0, "Contract": 0.0, "Dependents": 0.0, "DeviceProtection": 0.0, "InternetService": 0.0, "MonthlyCharges": 0.0, "MultipleLines": 0.0, "OnlineBackup": 0.0, "OnlineSecurity": 0.0, "PaperlessBilling": 0.0, "Partner": 0.0, "PaymentMethod": 0.0, "PhoneService": 0.0, "SeniorCitizen": 0.0, "StreamingMovies": 0.0, "StreamingTV": 0.0027513077, "TechSupport": 0.0, "TotalCharges": 0.000541107, "TotalCharges1": 0.0125408172, "customerID": 0.0, "gender": 1.0, "tenure": 0.0}, {"Churn": 0.0799659085, "Contract": 0.3236292241, "Dependents": 0.0081087683, "DeviceProtection": 0.0978905729, "InternetService": 0.0, "MonthlyCharges": 0.0197705524, "MultipleLines": 0.0791590375, "OnlineBackup": 0.0875949622, "OnlineSecurity": 0.0742741383, "PaperlessBilling": 0.0, "Partner": 0.0838970473, "PaymentMethod": 0.1089953339, "PhoneService": 0.0, "SeniorCitizen": 0.0, "StreamingMovies": 0.0586024461, "StreamingTV": 0.0561383592, "TechSupport": 0.0693137664, "TotalCharges": 0.5211348992, "TotalCharges1": 0.4932912137, "customerID": 0.0, "gender": 0.0, "tenure": 1.0}], "HasBasicStats": 1, "functionsApplied": [{"functionName": "formula", "applyOn": [{"columnName": "TotalCharges", "type": "real", "min": "18.8", "max": "8684.8", "mean": "2283.3"}, {"columnName": "MonthlyCharges", "type": "real", "min": "18.3", "max": "118.8", "mean": "64.8"}], "functionsData": [{"operand1": "TotalCharges", "operator": "add", "operand2": "MonthlyCharges", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "TotalCharges", "type": "real", "min": "18.8", "max": "8684.8", "mean": "2283.3"}}, {"Columndata": {"columnName": "MonthlyCharges", "type": "real", "min": "18.3", "max": "118.8", "mean": "64.8"}}, {"Columndata": ""}], "newColumnName": "TotalCharges1"}]}], "functionChanges": [{"columnName": "TotalCharges", "functionName": "Formula", "Type": "real", "Parameters": [{"operand1": "TotalCharges", "operator": "add", "operand2": "MonthlyCharges", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "TotalCharges", "type": "real", "min": "18.8", "max": "8684.8", "mean": "2283.3"}}, {"Columndata": {"columnName": "MonthlyCharges", "type": "real", "min": "18.3", "max": "118.8", "mean": "64.8"}}, {"Columndata": ""}], "newColumnName": "TotalCharges1"}]}, {"columnName": "MonthlyCharges", "functionName": "Formula", "Type": "real", "Parameters": [{"operand1": "TotalCharges", "operator": "add", "operand2": "MonthlyCharges", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "TotalCharges", "type": "real", "min": "18.8", "max": "8684.8", "mean": "2283.3"}}, {"Columndata": {"columnName": "MonthlyCharges", "type": "real", "min": "18.3", "max": "118.8", "mean": "64.8"}}, {"Columndata": ""}], "newColumnName": "TotalCharges1"}]}], "fileheader": [{"field": "customerID", "alias": "customerID", "generated": 0, "position": 1, "type": "String"}, {"field": "gender", "alias": "gender", "generated": 0, "position": 2, "type": "String"}, {"field": "SeniorCitizen", "alias": "SeniorCitizen", "generated": 0, "position": 3, "type": "numeric"}, {"field": "Partner", "alias": "Partner", "generated": 0, "position": 4, "type": "String"}, {"field": "Dependents", "alias": "Dependents", "generated": 0, "position": 5, "type": "String"}, {"field": "tenure", "alias": "tenure", "generated": 0, "position": 6, "type": "numeric"}, {"field": "PhoneService", "alias": "PhoneService", "generated": 0, "position": 7, "type": "String"}, {"field": "MultipleLines", "alias": "MultipleLines", "generated": 0, "position": 8, "type": "String"}, {"field": "InternetService", "alias": "InternetService", "generated": 0, "position": 9, "type": "String"}, {"field": "OnlineSecurity", "alias": "OnlineSecurity", "generated": 0, "position": 10, "type": "String"}, {"field": "OnlineBackup", "alias": "OnlineBackup", "generated": 0, "position": 11, "type": "String"}, {"field": "DeviceProtection", "alias": "DeviceProtection", "generated": 0, "position": 12, "type": "String"}, {"field": "TechSupport", "alias": "TechSupport", "generated": 0, "position": 13, "type": "String"}, {"field": "StreamingTV", "alias": "StreamingTV", "generated": 0, "position": 14, "type": "String"}, {"field": "StreamingMovies", "alias": "StreamingMovies", "generated": 0, "position": 15, "type": "String"}, {"field": "Contract", "alias": "Contract", "generated": 0, "position": 16, "type": "String"}, {"field": "PaperlessBilling", "alias": "PaperlessBilling", "generated": 0, "position": 17, "type": "String"}, {"field": "PaymentMethod", "alias": "PaymentMethod", "generated": 0, "position": 18, "type": "String"}, {"field": "MonthlyCharges", "alias": "MonthlyCharges", "generated": 0, "position": 19, "type": "real"}, {"field": "TotalCharges", "alias": "TotalCharges", "generated": 0, "position": 20, "type": "real"}, {"field": "Churn", "alias": "Churn", "generated": 0, "position": 21, "type": "String"}, {"field": "TotalCharges1", "alias": "TotalCharges1", "generated": 1, "position": 22, "type": "real"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run telcochurndatapipelineHooks.ipynb
try:
	#sinkPreExecutionHook()

	telcochurn = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/654d2a36bb5c80aef638770a/part-00000-6beb7129-06e5-4522-b34b-ae22f0d8b6be-c000.csv', 'samplecount': 691, 'originalcount': 7043, 'orignalKey': '/FileStore/platform/testdata/1695815396284_Telco_Customer_Churn_Train.csv', 'pathOnly': '/Retail And Marketing/Telco Churn', 'project_id': '6514163229e1b0f5334a66ed', 'parent_id': '6514163229e1b0f5334a66ed', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'customerID', 'alias': 'customerID', 'type': 'String', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'gender', 'alias': 'gender', 'type': 'String', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'SeniorCitizen', 'alias': 'SeniorCitizen', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Partner', 'alias': 'Partner', 'type': 'String', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Dependents', 'alias': 'Dependents', 'type': 'String', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'tenure', 'alias': 'tenure', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'PhoneService', 'alias': 'PhoneService', 'type': 'String', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'MultipleLines', 'alias': 'MultipleLines', 'type': 'String', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'InternetService', 'alias': 'InternetService', 'type': 'String', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'OnlineSecurity', 'alias': 'OnlineSecurity', 'type': 'String', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'OnlineBackup', 'alias': 'OnlineBackup', 'type': 'String', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'DeviceProtection', 'alias': 'DeviceProtection', 'type': 'String', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'TechSupport', 'alias': 'TechSupport', 'type': 'String', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'StreamingTV', 'alias': 'StreamingTV', 'type': 'String', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'StreamingMovies', 'alias': 'StreamingMovies', 'type': 'String', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Contract', 'alias': 'Contract', 'type': 'String', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'PaperlessBilling', 'alias': 'PaperlessBilling', 'type': 'String', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'PaymentMethod', 'alias': 'PaymentMethod', 'type': 'String', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'MonthlyCharges', 'alias': 'MonthlyCharges', 'type': 'real', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'TotalCharges', 'alias': 'TotalCharges', 'type': 'real', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Churn', 'alias': 'Churn', 'type': 'String', 'position': '20'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'TotalCharges1', 'alias': 'TotalCharges1', 'type': 'real', 'position': '22'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'customerID', 'alias': 'customerID', 'type': 'String', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'gender', 'alias': 'gender', 'type': 'String', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'SeniorCitizen', 'alias': 'SeniorCitizen', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Partner', 'alias': 'Partner', 'type': 'String', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Dependents', 'alias': 'Dependents', 'type': 'String', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'tenure', 'alias': 'tenure', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'PhoneService', 'alias': 'PhoneService', 'type': 'String', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'MultipleLines', 'alias': 'MultipleLines', 'type': 'String', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'InternetService', 'alias': 'InternetService', 'type': 'String', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'OnlineSecurity', 'alias': 'OnlineSecurity', 'type': 'String', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'OnlineBackup', 'alias': 'OnlineBackup', 'type': 'String', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'DeviceProtection', 'alias': 'DeviceProtection', 'type': 'String', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'TechSupport', 'alias': 'TechSupport', 'type': 'String', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'StreamingTV', 'alias': 'StreamingTV', 'type': 'String', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'StreamingMovies', 'alias': 'StreamingMovies', 'type': 'String', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Contract', 'alias': 'Contract', 'type': 'String', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'PaperlessBilling', 'alias': 'PaperlessBilling', 'type': 'String', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'PaymentMethod', 'alias': 'PaymentMethod', 'type': 'String', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'MonthlyCharges', 'alias': 'MonthlyCharges', 'type': 'real', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'TotalCharges', 'alias': 'TotalCharges', 'type': 'real', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Churn', 'alias': 'Churn', 'type': 'String', 'position': '20'}], 'server': 'https://ml.numtra.com:443', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'TelcoChurnTransformed.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-891b5564-3730-4d64-94f5-8d11fb0c115b-c000.csv', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
